# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

Refer - https://www.kaggle.com/code/melihkanbay/knn-best-parameters-gridsearchcv

In [44]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [45]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()


# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)
# Bind the 'class' (wine target) to the DataFrame
wine_df.columns
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df.describe
rows,cols = wine_df.shape


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [46]:
# Your answer here
print("Rows in data set :", rows) # number of rows in data set

Rows in data set : 178


_(ii)_ How many variables (columns) does the dataset contain?

In [47]:
# Your answer here
print(wine_df.columns) # list all columns
print("Columns in data set : ", cols) # total columns in data set


Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'class'],
      dtype='object')
Columns in data set :  14


_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [48]:
# Your answer here
print("'class' data type : ",wine_df.dtypes['class']) # data type for class response variable

print("Unique values in 'class' column: ",wine_df["class"].unique()) # unique values for response variable 'class'

'class' data type :  int64
Unique values in 'class' column:  [0 1 2]



_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [49]:
# Your answer here
#X = wine_df.iloc[:, :-1]
print("Total number of predictor variables in data set :",wine_df.iloc[:, :-1].shape[1]) # total number of predictor variables in data set
#print(X.columns) 

Total number of predictor variables in data set : 13


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [50]:
# self exploration of data
wine_df.describe
wine_df.isna().sum

<bound method DataFrame.sum of      alcohol  malic_acid    ash  alcalinity_of_ash  magnesium  total_phenols  \
0      False       False  False              False      False          False   
1      False       False  False              False      False          False   
2      False       False  False              False      False          False   
3      False       False  False              False      False          False   
4      False       False  False              False      False          False   
..       ...         ...    ...                ...        ...            ...   
173    False       False  False              False      False          False   
174    False       False  False              False      False          False   
175    False       False  False              False      False          False   
176    False       False  False              False      False          False   
177    False       False  False              False      False          False   

     fla

In [51]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

**Answer**

To ensure fair contribution from all variables, we often scale and center data before
entering it into the model. This process, called standardization, adjusts the data so each variable has a mean of 0 and a standard deviation of 1, allowing the model to consider each variable equally based on its relationship to the outcome rather than its scale.



(ii) Why did we elect not to standard our response variable `Class`?

**Answer**

Standardizing response variable can complicate interpretation and obscure important information, which is why it is often avoided in traditional statistical analysis.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

**Answer**

 Setting the same seed value guarantees the same results each time the analysis is run. Different seed values result in different random patterns, but the same value ensures consistent results.



(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [52]:
# Do not touch
np.random.seed(123)
# Create a random vector of True and False values to split the data
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])
#print(split)
# Your code here...
# Define the training set for X (predictors)
training_X = predictors_standardized[split]
# Define the testing set for X (predictors)
testing_X = predictors_standardized[~split]

# Define the training set for Y (response)
training_Y = wine_df.loc[split, 'class']
# Define the testing set for Y (response)
testing_Y = wine_df.loc[~split, 'class']

print("Training  data set shapes:", training_X.shape, training_Y.shape)
print("Testing set shapes:", testing_X.shape, testing_Y.shape)



Training  data set shapes: (140, 13) (140,)
Testing set shapes: (38, 13) (38,)


#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [53]:
# Your code here...

knn = KNeighborsClassifier() # initializing knn classifier
k_range = list(range(1, 50)) # grid range from 1 to 50
param_grid = dict(n_neighbors=k_range)
#print(param_grid)
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False) # grid search
grid.fit(training_X, training_Y) # model fitting on training data
best_n_neighbors = grid.best_params_['n_neighbors'] # best value of n_neighors

print(f"The best value of n_neighbors is : {best_n_neighbors}")

The best value of n_neighbors is : 8


#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [55]:
# Your code here...
knn_opt = KNeighborsClassifier(n_neighbors=best_n_neighbors) # Initialize KNN classifier with the best n_neighbors
knn_opt.fit(training_X, training_Y) # Train the model on the entire training set
y_pred = knn_opt.predict(testing_X) # Make predictions on the test set
accuracy = accuracy_score(testing_Y, y_pred) #Calculate and display the accuracy score
print(f"The accuracy of test set is: {accuracy:.4f}")

The accuracy of test set is: 0.9474


# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applying_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-4-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
